In [20]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os, glob

DATA_DIR = './data/icon/'
FEATURES_DIR = './output/'
VALID_EXTENSION = ['jpg', 'jpeg', 'JPG', 'JPEG', 'png', 'PNG', 'Png']

%matplotlib inline

In [21]:
class net(nn.Module):
    
    def __init__(self):
        super(net, self).__init__()
        self.net = models.resnet50(pretrained=True)
    
    def forward(self, input):
        output = self.net.conv1(input)
        output = self.net.bn1(output)
        output = self.net.relu(output)
        output = self.net.maxpool(output)
        output = self.net.layer1(output)
        output = self.net.layer2(output)
        output = self.net.layer3(output)
        output = self.net.layer4(output)
        output = self.net.avgpool(output)
        return output


In [22]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        VGG = models.vgg16(pretrained=True)
        self.feature = VGG.features
        self.classifier = nn.Sequential(*list(VGG.classifier.children())[:-3])
        pretrained_dict = VGG.state_dict()
        model_dict = self.classifier.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.classifier.load_state_dict(model_dict)
 
    def forward(self, x):
        output = self.feature(x)
        output = output.view(output.size(0), -1)
        output = self.classifier(output)
        return output

In [23]:
model = net()
# model = Encoder()
# model = model.cuda()



In [24]:
def extractor(img_path, saved_path, net, use_gpu):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()]
    )
    
    img = Image.open(img_path)
    img = transform(img)
    print('img shape: ', img.shape)
 
    x = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False)
    print('x shape: ', x.shape)
 
    if use_gpu:
        x = x.cuda()
        net = net.cuda()
    y = net(x).cpu()
    y = torch.squeeze(y)
    y = y.data.numpy()
    print(y.shape)
    np.savetxt(saved_path, y, delimiter=',')

In [46]:
def preprocessImage(img_path, save_dir):
    
    img = Image.open(img_path)
#     img = img.convert('RGB')
#     file_name = img_path.split('/')[-1]
#     save_path = os.path.join(save_dir, file_name)
#     img.save(save_path)
    
    tensorTransformer = transforms.ToTensor()
    tensorImg = tensorTransformer(img)
    print('path:{},  image shape: {}'.format(img_path, tensorImg.shape))
    if img_path == './data/icon/c11_1.png':
        print(tensorImg)

In [47]:
x = os.walk(DATA_DIR)
for path, dirctory, filelist in x:
    for filename in filelist:
        file_glob = os.path.join(path, filename)
        is_valid = False
        for ext in VALID_EXTENSION:
            if file_glob.endswith(ext):
                is_valid = True
                break
        if is_valid:
            preprocessImage(file_glob, FEATURES_DIR)
#             print(file_glob)

path:./data/icon/c6_2.png,  image shape: torch.Size([4, 10, 10])
path:./data/icon/c4_1.png,  image shape: torch.Size([4, 39, 39])
path:./data/icon/c6_3.png,  image shape: torch.Size([1, 32, 32])
path:./data/icon/c6_1.png,  image shape: torch.Size([4, 12, 12])
path:./data/icon/c4_2.png,  image shape: torch.Size([4, 20, 24])
path:./data/icon/c2_1.png,  image shape: torch.Size([1, 64, 64])
path:./data/icon/c2_2.png,  image shape: torch.Size([4, 64, 64])
path:./data/icon/c7_1.png,  image shape: torch.Size([1, 22, 15])
path:./data/icon/c5_2.png,  image shape: torch.Size([4, 16, 18])
path:./data/icon/c7_2.png,  image shape: torch.Size([4, 16, 18])
path:./data/icon/c7_3.png,  image shape: torch.Size([4, 20, 22])
path:./data/icon/c5_1.png,  image shape: torch.Size([1, 48, 39])
path:./data/icon/c1_1.png,  image shape: torch.Size([4, 64, 64])
path:./data/icon/c9_8.png,  image shape: torch.Size([4, 48, 48])
path:./data/icon/c7_7.png,  image shape: torch.Size([4, 16, 16])
path:./data/icon/c7_6.png

In [35]:
file_list = []
x = os.walk(DATA_DIR)
for path, directory, filelist in x:
#     print(path)
#     print(directory)
#     print(filelist)
#     print('---')
    for filename in filelist:
        file_glob = os.path.join(path, filename)
        file_list.extend(glob.glob(file_glob))
        
# print(file_list)

use_gpu = torch.cuda.is_available()
for index, x_path in enumerate(file_list):
    file_name = x_path.split('/')[-1]
    fx_path = os.path.join(FEATURES_DIR, file_name + '.txt')
    print(fx_path)
    extractor(x_path, fx_path, model, use_gpu)

./output/c6_2.png.txt
img shape:  torch.Size([4, 224, 224])
x shape:  torch.Size([1, 4, 224, 224])


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 4, 224, 224] to have 3 channels, but got 4 channels instead